# Lunar Cycle Influence on Twitter Sentiment and Stock Markets

article on lunar cycles & crime: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1444800/

project proposal write-up: https://docs.google.com/document/d/1sAl3SyxhI6gLZ1zDEHhtIOfjB0FzofjjM3Bp7CaIBgM/edit#

#### Obtain Scrub Explore Model iNterpret!

## Obtaining Twitter Data

instructions on accessing the Twitter API and performing sentiment analysis: https://www.geeksforgeeks.org/twitter-sentiment-analysis-using-python/

this one has more options and discusses removing user handles and more: https://www.analyticsvidhya.com/blog/2018/07/hands-on-sentiment-analysis-dataset-python/

lunar phases: https://www.calendar-12.com/moon_phases/2018

   - I need to specify that tweets should come from the middle of the day (when stock market is open)
       - if this is not fruitful, I can specify a time after sunset for all time zones
   - I also need to limit my search to English language
   - I need to limit my search to the United States (or even East Coast or NYC/Chicago, depending on size of dataset)
    
    

In [ ]:
# Sentiment analysis code below comes from:
# https://www.geeksforgeeks.org/twitter-sentiment-analysis-using-python/

import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 
  
class TwitterClient(object): 
    ''' 
    Generic Twitter Class for sentiment analysis. 
    '''
    def __init__(self): 
        ''' 
        Class constructor or initialization method. 
        '''
        # keys and tokens from the Twitter Dev Console 
        consumer_key = 'XXXXXXXXXXXXXXXXXXXXXXXX'
        consumer_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXX'
        access_token = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXX'
        access_token_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXX'
  
        # attempt authentication 
        try: 
            # create OAuthHandler object 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            self.auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed") 
  
    def clean_tweet(self, tweet): 
        ''' 
        Utility function to clean tweet text by removing links and
        special characters using simple regex statements. 
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) 
                                    |(\w+:\/\/\S+)", " ", tweet).split()) 
  
    def get_tweet_sentiment(self, tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(self.clean_tweet(tweet)) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'
  
    def get_tweets(self, query, count=1000): 
        ''' 
        Main function to fetch tweets and parse them. 
        '''
        # empty list to store parsed tweets 
        tweets = [] 
  
        try: 
            # call twitter api to fetch tweets 
            fetched_tweets = self.api.search(q = query, count = count) 
  
            # parsing tweets one by one 
            for tweet in fetched_tweets: 
                # empty dictionary to store required params of a tweet 
                parsed_tweet = {} 
  
                # saving text of tweet 
                parsed_tweet['text'] = tweet.text 
                # saving sentiment of tweet 
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 
  
                # appending parsed tweet to tweets list 
                if tweet.retweet_count > 0: 
                    # if tweet has retweets, ensure that it is appended only once 
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet) 
  
            # return parsed tweets 
            return tweets 
  
        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e)) 
  
def main(query, count=1000): 
    # creating object of TwitterClient Class 
    api = TwitterClient() 
    # calling function to get tweets 
    tweets = api.get_tweets(query = query, count = count) 
  
    # picking positive tweets from tweets 
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
    # percentage of positive tweets 
    print(f"Positive tweets percentage: {100*len(ptweets)/len(tweets)} %") 
    # picking negative tweets from tweets 
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
    # percentage of negative tweets 
    print(f"Negative tweets percentage: {100*len(ntweets)/len(tweets)} %") 
    # percentage of neutral tweets 
    print(f"Neutral tweets percentage: {100*len(tweets - ntweets - ptweets)/len(tweets)} %") 
  
    # printing first 5 positive tweets 
    print("\n\nPositive tweets:") 
    for tweet in ptweets[:10]: 
        print(tweet['text']) 
  
    # printing first 5 negative tweets 
    print("\n\nNegative tweets:") 
    for tweet in ntweets[:10]: 
        print(tweet['text']) 
  
# if __name__ == "__main__": 
#     # calling main function 
#     main() 

## Obtaining Financial Data

website to get historical infra-day stock prices: https://www.alphavantage.co/documentation/
   - request limitations: 5/minute, 500/day
   - URL structure: https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&outputsize=full&apikey=demo
   - (back up option: https://docs.intrinio.com/tutorial/web_api)
   
I need to be careful about how I look at this data. Since stock markets are only open 6.5 hours a day, 5 days a week, certain lunar cycle days will fall on weekends, while considerable after-hours trading can affect next-day opening prices. Since lunar cycles are most noticeble at night, I should probably set stock days as 24-hour periods, from opening bell to following opening bell to account for evening trading. 

In [1]:
import json
import requests
import pandas as pd

# HOW DO I HIDE my API keys from the public and still use the code???

In [2]:
# Entering the Stock Symbol and API key
symbol = 'MSFT'
stocks_API_key = ''

In [3]:
# Making an API request for a certain stock's hsitory
credentials = {'function':'TIME_SERIES_DAILY', 'symbol':symbol, 'outputsize':'full', 'apikey':stocks_API_key}
r = requests.get('https://www.alphavantage.co/query', params=credentials)
print(r.status_code)
r.status_code == requests.codes.ok

200


True

In [24]:
# stock_df = pd.DataFrame(r.text)
df = pd.DataFrame(r.json()["Time Series (Daily)"])
df = df.T.reset_index()
df.columns = ['date','open','high','low','close','volume']
df.date = pd.to_datetime(df.date)
df[['open','high','low','close','volume']] = df[['open','high','low','close','volume']].astype(float)
df.head()

,index,1. open,2. high,3. low,4. close,5. volume
0,2019-09-17,136.9500,137.4900,136.4250,136.9000,6552343
1,2019-09-16,135.8300,136.7000,135.6600,136.3300,14785072
2,2019-09-13,137.7800,138.0600,136.5700,137.3200,22902300
3,2019-09-12,137.8500,138.4200,136.8700,137.5200,27010000
4,2019-09-11,135.9100,136.2700,135.0900,136.1200,24726100


## Obtaining List of Dates of 2018 Lunar Phases

file is stored as `lunar_phases_2018.csv`

In [39]:
# loading in a list of 2018 lunar phases
phases_2018 = pd.read_csv('lunar_phases_2018.csv')
phases_2018.date = pd.to_datetime(phases_2018.date)
phases_2018.head()

,phase,date,weekday,time
0,Full Moon,2018-01-01,Mon,20:25:00
1,Last Quarter,2018-01-08,Mon,16:26:00
2,New Moon,2018-01-16,Tue,20:18:00
3,First Quarter,2018-01-24,Wed,16:20:00
4,Full Moon,2018-01-31,Wed,7:27:00
